# 협업 필터링(Collaborative Filtering)

## 데이터 불러오기

In [1]:
import pandas as pd

review_df = pd.read_csv('../data/ulsan_rest_review_id.csv')
title_df = pd.read_csv('../data/ulsan_rest_title.csv')

In [2]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6453 entries, 0 to 6452
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    6453 non-null   object
 1   u_id     6453 non-null   int64 
 2   user_id  6432 non-null   object
 3   score    6453 non-null   int64 
 4   comment  5247 non-null   object
dtypes: int64(2), object(3)
memory usage: 252.2+ KB


## 데이터 전처리

In [3]:
print(review_df.isnull().sum())

review_df.dropna(axis=0, how='any', subset=['user_id'], inplace=True)

title         0
u_id          0
user_id      21
score         0
comment    1206
dtype: int64


In [4]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6432 entries, 0 to 6431
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    6432 non-null   object
 1   u_id     6432 non-null   int64 
 2   user_id  6432 non-null   object
 3   score    6432 non-null   int64 
 4   comment  5228 non-null   object
dtypes: int64(2), object(3)
memory usage: 301.5+ KB


In [5]:
# CountStatus = pd.value_counts(review_df['user_id'], sort=True)
# CountStatus.plot.bar()

In [6]:
table_df = pd.merge(review_df, title_df, on='title', how='inner')
table_df

,title,u_id,user_id,score,comment,p_id,rate,tags,address,url
0,바다바라기 본점,0,0,1,인생최악의 킹크랩 이였습니다. 게의 상태가 안좋은 건지 잘못찐건지.킹크랩의 고유의 ...,420,4.0,"게,대게","울산 북구 정자1길 111 1,2층 (우)44233",https://place.map.kakao.com/15261734
1,바다바라기 본점,21,-,1,진짜 돈아깝다음식이랑 직원한테 투자해야할걸 sns 광고비에만 몰빵하는듯어딜가든 여기...,420,4.0,"게,대게","울산 북구 정자1길 111 1,2층 (우)44233",https://place.map.kakao.com/15261734
2,바다바라기 본점,22,--,5,대게가 엄청큰데 살이 완전 가득 채워져있고 살이 부들부들해서 진짜 계속 먹을 수 있...,420,4.0,"게,대게","울산 북구 정자1길 111 1,2층 (우)44233",https://place.map.kakao.com/15261734
3,바다바라기 본점,33,(알 수 없음),5,평점이 좋길래 믿고 방문했어요대게도 신선하고 직원분들도 친절해서 기분 좋은 식사했네요,420,4.0,"게,대게","울산 북구 정자1길 111 1,2층 (우)44233",https://place.map.kakao.com/15261734
4,바다바라기 본점,39,.,5,울산여행와서 맛집검색해서 왔는데 맛있었습니다. #맛집 #성공적,420,4.0,"게,대게","울산 북구 정자1길 111 1,2층 (우)44233",https://place.map.kakao.com/15261734
...,...,...,...,...,...,...,...,...,...,...
6816,장군참치,3008,ㅈ,1,오마이갓 참치집이 아님 종업사기꾼,310,3.0,참치회,울산 남구 정동로80번길 19 (우)44712,https://place.map.kakao.com/2008161945
6817,도드람마을,3058,재민맘-김혜원,5,고기 육질이 좋네요~ 역시 차가 많은 이유가 있네용~ ^^,380,3.5,"육류,고기",울산 북구 진장로 85 (우)44250,https://place.map.kakao.com/26345203
6818,도드람마을,3140,정원,1,고기 골라 손에 쥐고 들어갔더니 안내를 안해줍니다.아무 자리에 가서 앉았더니 한참 ...,380,3.5,"육류,고기",울산 북구 진장로 85 (우)44250,https://place.map.kakao.com/26345203
6819,도드람마을,3405,"최고,최악만 메모하는 유니",5,상차림반찬들+고기질 다 너무좋아요북구사람이면 다 아는 넘나유명한곳직원분들도 다 친절하시고,380,3.5,"육류,고기",울산 북구 진장로 85 (우)44250,https://place.map.kakao.com/26345203


In [13]:
data_df = table_df[['p_id', 'u_id', 'score']]
data_df.tail(10)

,p_id,u_id,score
6811,26,3076,5
6812,106,2534,1
6813,157,2679,2
6814,157,3204,5
6815,310,2716,5
6816,310,3008,1
6817,380,3058,5
6818,380,3140,1
6819,380,3405,5
6820,380,3764,3


In [18]:
# import numpy as np

# n_places = np.max(data_df['p_id'])
# n_users = np.max(data_df['u_id'])
# shape = (n_users+1, n_places+1)
# shape

(3869, 535)

In [19]:
# adj_matrix = np.ndarray(shape, dtype=int)
# for _, row in data_df.iterrows():
#     adj_matrix[row['u_id']][row['p_id']] = row['score']

# adj_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Surprise 라이브러리 활용

In [22]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

In [39]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(data_df[['u_id', 'p_id', 'score']], reader)

### KNN

In [52]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5440  1.5006  1.5603  1.4867  1.5373  1.5258  0.0276  
MAE (testset)     1.2649  1.2116  1.2723  1.1972  1.2521  1.2396  0.0298  
Fit time          0.17    0.18    0.17    0.16    0.16    0.17    0.01    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    


{'test_rmse': array([1.54397793, 1.50056128, 1.56033177, 1.48673228, 1.53729628]),
 'test_mae': array([1.26485256, 1.211563  , 1.27226429, 1.19722935, 1.25210923]),
 'fit_time': (0.1668553352355957,
  0.1807098388671875,
  0.16567134857177734,
  0.1647017002105713,
  0.16141653060913086),
 'test_time': (0.018937349319458008,
  0.0184323787689209,
  0.01855921745300293,
  0.01868438720703125,
  0.017160415649414062)}

### SVD

In [41]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3817  1.3995  1.3955  1.3834  1.3829  1.3886  0.0074  
MAE (testset)     1.1211  1.1571  1.1579  1.1527  1.1357  1.1449  0.0143  
Fit time          0.10    0.09    0.09    0.08    0.07    0.08    0.01    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([1.38169625, 1.39951399, 1.39547163, 1.38341265, 1.38289032]),
 'test_mae': array([1.12113856, 1.15707837, 1.15790014, 1.15269229, 1.13574935]),
 'fit_time': (0.09841036796569824,
  0.09079766273498535,
  0.08724069595336914,
  0.07637786865234375,
  0.06716394424438477),
 'test_time': (0.01283574104309082,
  0.009602546691894531,
  0.009607791900634766,
  0.008535146713256836,
  0.006993770599365234)}

### NMF

In [42]:
model = NMF()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4913  1.4963  1.4973  1.4993  1.5348  1.5038  0.0157  
MAE (testset)     1.1973  1.2129  1.2100  1.2292  1.2479  1.2195  0.0175  
Fit time          0.30    0.32    0.30    0.27    0.26    0.29    0.02    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([1.49134723, 1.49625792, 1.49730402, 1.49931474, 1.53479394]),
 'test_mae': array([1.19731819, 1.21289426, 1.21004864, 1.22918876, 1.24786189]),
 'fit_time': (0.3028252124786377,
  0.31803441047668457,
  0.3030424118041992,
  0.2672920227050781,
  0.2593708038330078),
 'test_time': (0.007559776306152344,
  0.00836634635925293,
  0.00849008560180664,
  0.00697636604309082,
  0.0065076351165771484)}

### SVD++

In [43]:
model = SVDpp()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4097  1.3732  1.4108  1.3872  1.3722  1.3906  0.0169  
MAE (testset)     1.1582  1.1282  1.1662  1.1480  1.1435  1.1488  0.0130  
Fit time          0.18    0.15    0.15    0.15    0.14    0.15    0.01    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    


{'test_rmse': array([1.40966234, 1.3732118 , 1.41078979, 1.38723289, 1.37219004]),
 'test_mae': array([1.15822447, 1.12821451, 1.16620555, 1.14802593, 1.14348003]),
 'fit_time': (0.17633914947509766,
  0.1516246795654297,
  0.14712762832641602,
  0.15126252174377441,
  0.13931965827941895),
 'test_time': (0.0171964168548584,
  0.01740288734436035,
  0.01709461212158203,
  0.01703810691833496,
  0.016169071197509766)}

## GridSearch 활용한 최적 HP 탐색

In [50]:
from surprise.model_selection import GridSearchCV

params = {'k': range(3, 30, 1),	# 이웃 크기
          'sim_options': {'name' : ['pearson_baseline', 'cosine'],	# 유사도 지표
                          'user_based' : [True, False]}}	# UBCF(True), IBCF(False)

gs = GridSearchCV(KNNBasic, params, measures=['rmse', 'mae'], cv=5)
gs.fit(data, verbose=False)

print('최적의 조합', gs.best_params['rmse'])
print('최적의 조합의 RMSE', gs.best_score['rmse'])

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [44]:
from surprise.model_selection import GridSearchCV
params = {'n_epochs': [20, 40, 60, 100],	# 연산 반복 횟수
          'lr_all': [0.005, 0.008, 0.001],	# 학습률
          'n_factors': [5, 10, 30, 50, 100], # 잠재요인
}

gs = GridSearchCV(SVD, params, measures=['rmse', 'mae'], cv=5)
gs.fit(data)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.3637826951204377
{'n_epochs': 100, 'n_factors': 50}


In [47]:
params = {'n_epochs': [20, 40, 60, 100],	# 연산 반복 횟수
          'lr_all': [0.005, 0.008, 0.001],	# 학습률
          'n_factors': [5, 10, 30, 50, 100], # 잠재요인
}


gs = GridSearchCV(SVDpp, params, measures=['rmse', 'mae'], cv=5)
gs.fit(data)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.3766799168639219
{'n_epochs': 20, 'lr_all': 0.008, 'n_factors': 5}
